### Steps for ROI Extraction
We will start with adjusting the color space
From some EDA, I have found LAB, LUV and YUV colors spaces to be useful for ROI extraction task

Here's the current pipeline
- Convert RGB image into another color space (possibly LAB)
- Extract L channel and apply enhancement using `convertScaleAbs` function from cv2 [alpha = 2.0, beta=-50]
- Dilate image to increase ROI area
- Apply binary threshold on the image with threshold value of 200
- Applying `findContours` on the thresholded image
- Fetch each contours area and sort in descending order.
- Pick highest contour area objects and create bounding boxes using `boundingRect`
- These will be used as ROIs for extraction of hot liquid

- Need to apply some kind of technique to identify how many objects are there in the frame. 
    - Also, how do we know the ROI is actually a coffee mug or glass or something else.
        Template Matching, or maybe some other technique. 
            Some kind of edge detection


In [ ]:
import cv2
from PIL import Image
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import imageio as io
import pandas as pd
import yaml
import sys
from tqdm.notebook import tqdm_notebook
from matplotlib import patches
from pycocotools import mask as M
from pprint import pprint
sys.path.append('../scripts')
import utils

In [ ]:
with open('../config.yaml','r') as f:
    config = yaml.safe_load(f)
    
img_paths = sorted(glob.glob(config['new_dataset']['images']+'/img_thermal_*'))

In [ ]:
color_convs = {
    'HLS':cv2.COLOR_RGB2HLS,
    'HSV': cv2.COLOR_RGB2HSV,
    'LAB':cv2.COLOR_RGB2LAB,
    'LUV': cv2.COLOR_RGB2LUV,
    'YUV': cv2.COLOR_RGB2YUV
}

In [ ]:
def get_enhanced_image(image_path, apply_ce=False, apply_blur=False, apply_clahe=False, apply_dilation=False):
    c_img = utils.rotate_to_vertical(image_path)
    og_img = c_img
    c_img = cv2.cvtColor(c_img, cv2.COLOR_RGB2GRAY)
    
    if apply_clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(32,32))
        c_img = clahe.apply(c_img)
    
    if apply_ce:
        c_img = cv2.equalizeHist(c_img)
    
    if apply_blur:
        c_img = cv2.GaussianBlur(c_img, (51, 51), 15)

    cvt_img = cv2.cvtColor(og_img, cv2.COLOR_RGB2LAB)
    # cvt_img = og_img
    # img_dilate = None
    
    enhanced_img = cv2.convertScaleAbs(cvt_img[...,0], alpha=2,beta=-70)
    if apply_dilation:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Adjust kernel size as needed
        img_dilate = cv2.dilate(enhanced_img, kernel, iterations=2)
    
    return c_img, og_img, cvt_img, enhanced_img, img_dilate

In [ ]:
def get_threshold_img(img, t_value=200, apply_open=False, apply_erode_n_dilate=False):
    _, img = cv2.threshold(img, t_value, 255, cv2.THRESH_BINARY)
    
    if apply_open:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))  # Adjust kernel size as needed
        opened = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        img = opened
        
    if apply_erode_n_dilate:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6))  # Adjust kernel size as needed
        eroded = cv2.erode(img, kernel, iterations=6)
        dilated = cv2.dilate(eroded, kernel, iterations=4)
        img = dilated

    return img

In [ ]:
def get_contours_n_bbox(img):
    contours, _ = cv2.findContours(img.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    new_contours = [np.squeeze(c, axis=1) for c in contours]

    contour_areas = pd.DataFrame([(int(i), cv2.contourArea(c)) for i, c in enumerate(contours)])
    contour_areas.columns = ['index', 'area']
    contour_areas.sort_values('area', ascending=False, inplace=True)
    contour_areas['index'] = contour_areas['index'].astype(int)
    
    bboxes = [cv2.boundingRect(new_contours[int(row[0])]) for idx, row in enumerate(contour_areas.iloc[:4].values)]
    
    return bboxes

## PATTERN MATCHING MISSING!
important for identifying if the identified ROI is of a glass/mug or something else hot
    - might have to make use of other channels to see if liquid is in the mug and is possibly less than the height of the cup.

In [ ]:
n = img_paths.__len__()  # total number of images
grid_shape = (int(np.sqrt(n))+1, int(np.sqrt(n))+1)
fig, axs = plt.subplots(nrows=grid_shape[0], ncols=grid_shape[1], figsize=(25, 25))

axs = axs.flatten()

for i, img_p in tqdm_notebook(enumerate(img_paths)):
    c_img, og_img, cvt_img, enhanced_img, img_dilate = get_enhanced_image(image_path=img_p, apply_dilation=True, apply_blur=True, apply_ce=True)
    t_img = get_threshold_img(img_dilate, apply_open=True, apply_erode_n_dilate=True)
    bboxes = get_contours_n_bbox(t_img)
    
    ax = axs[i]

    ax.imshow(img_dilate)
    # ax.imshow(t_img, alpha=0.5)
    for bbox in bboxes:
        ax.add_patch(patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], facecolor='none', edgecolor='white'))
    
    ax.set_title(i)
    ax.axis('off')

for ax in axs[n:]:
    ax.remove()

plt.show()

In [ ]:
from random import randint

idx = randint(0, img_paths.__len__()-1)
# idx = 60
print(idx)
og_img = utils.rotate_to_vertical(img_paths[idx])
# red = og_img[...,0]
# red = cv2.GaussianBlur(red, (11, 11), 5)
# red = (red > 4).astype(np.uint8)

def gamma_correction(image, gamma):
  image = image / 255.0
  corrected = np.power(image, gamma)
  return np.uint8(corrected * 255.0)


red_green = og_img[...,:2].mean(axis=-1)
red_green = cv2.GaussianBlur(red_green, (11, 11), 5)
# plt.imshow(red_green)

gamma_rg = gamma_correction(red_green, gamma=2.5)
gamma_rg = gamma_rg > 20
_, labels, bboxes, _ = cv2.connectedComponentsWithStats(gamma_rg.astype(np.uint8), connectivity=8)

# plt.imshow(og_img, cmap='gray', alpha=0.5)
for box in bboxes[1:]:
    x,y,w,h,area = box
    if area > 5000:
        # plt.gca().add_patch(patches.Rectangle((x-5,y),w+10,h,facecolor='none',edgecolor='white'))
        lx1 = x + w
        lx2 = x + h + w
        og_w, og_h = og_img.shape[:2]
        ly = y + h
        # plt.gca().add_patch(patches.Rectangle((lx1,ly),lx2-lx1,og_h-ly,facecolor='none',edgecolor='red'))
        # og_img[ly:og_img.shape[0],x-15:x+w+15,:] = 0
        og_img[ly:og_img.shape[0],x-15:x+w+15,:] = 0
        # plt.gca().add_patch(patches.Rectangle((x-5,ly),w+10,og_h,facecolor='none',edgecolor='red'))
plt.imshow(og_img, cmap='gray')
plt.show()

In [ ]:
n = img_paths.__len__()  # total number of images
grid_shape = (int(np.sqrt(n))+1, int(np.sqrt(n))+1)
fig, axs = plt.subplots(nrows=grid_shape[0], ncols=grid_shape[1], figsize=(25, 25))

axs = axs.flatten()

for i, img_p in tqdm_notebook(enumerate(img_paths)):
    og_img = utils.rotate_to_vertical(img_p)
    
    # c_img, og_img, cvt_img, enhanced_img, img_dilate = get_enhanced_image(image_path=img_p, apply_dilation=True, apply_blur=True, apply_ce=True)
    # t_img = get_threshold_img(img_dilate, apply_open=True, apply_erode_n_dilate=True)
    # bboxes = get_contours_n_bbox(t_img)
    
    ax = axs[i]
    red = og_img[...,0]
    red = cv2.GaussianBlur(red, (11, 11), 5)
# blurred = cv2.GaussianBlur(enhanced_img, (11, 11), 5)
    # axs[1].imshow(blurred)
    red = (red > 4).astype(np.uint8)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 3))  # Adjust kernel size as needed
    eroded = cv2.erode(red, kernel, iterations=5)
    
    ax.imshow(red)
    # cv2.connectedComponentsWithStats(red, connectivity=4)   
    # ax.imshow(eroded, cmap='gray', alpha=0.5)
    # ax.imshow(t_img, alpha=0.5)
    # for bbox in bboxes:
    #     ax.add_patch(patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], facecolor='none', edgecolor='white'))
    
    ax.set_title(i)
    ax.axis('off')

for ax in axs[n:]:
    ax.remove()

plt.show()

In [ ]:
def get_enhanced_image_2(img, apply_ce=False, apply_blur=False, apply_clahe=False, apply_dilation=False):
    # c_img = utils.rotate_to_vertical(image_path)
    og_img = img
    c_img = cv2.cvtColor(og_img, cv2.COLOR_RGB2GRAY)
    
    if apply_clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(32,32))
        c_img = clahe.apply(c_img)
    
    if apply_ce:
        c_img = cv2.equalizeHist(c_img)
    
    if apply_blur:
        c_img = cv2.GaussianBlur(c_img, (51, 51), 15)


    cvt_img = cv2.cvtColor(og_img, cv2.COLOR_RGB2LAB)
    # cvt_img = og_img
    # img_dilate = None
    
    enhanced_img = cv2.convertScaleAbs(cvt_img[...,0], alpha=2,beta=-70)
    if apply_dilation:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Adjust kernel size as needed
        img_dilate = cv2.dilate(enhanced_img, kernel, iterations=2)
    
    return c_img, og_img, cvt_img, enhanced_img, img_dilate

In [ ]:
# c_img = cv2.equalizeHist(og_img[...,-1])
fig, axs = plt.subplots(1,5, figsize=(16,5))
for i in range(4):
    axs[i].axis('off')

c_img, og_img, cvt_img, enhanced_img, img_dilate = get_enhanced_image_2(og_img, apply_dilation=True)


e_img = cv2.convertScaleAbs(og_img[...,[1,2]].mean(axis=-1), alpha=2,beta=-90)
axs[0].imshow(enhanced_img)

# blurred = cv2.GaussianBlur(og_img.mean(axis=-1).astype(np.uint8), (11, 11), 5)
# blurred = cv2.GaussianBlur(og_img[...,[0,2]].mean(axis=-1).astype(np.uint8), (11, 11), 5)
blurred = cv2.GaussianBlur(enhanced_img, (11, 11), 5)
# blurred = cv2.GaussianBlur(enhanced_img, (11, 11), 5)
axs[1].imshow(blurred)
# thresh = cv2.adaptiveThreshold(enhanced_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,31, 5)
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51, 5)
axs[2].imshow(thresh)

# kernel = np.ones((7, 7), np.uint8)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))  # Adjust kernel size as needed
eroded = cv2.erode(thresh, kernel, iterations=2)
axs[3].imshow(eroded)

# opening = cv2.morphologyEx(thresh[thresh.shape[0] // 2:,...], cv2.MORPH_OPEN, kernel, iterations=1)
# new_thresh = thresh
# new_thresh[thresh.shape[0] // 2:,...] = opening
# axs[3].imshow(new_thresh)
# edges = cv2.Canny(eroded, 200, 255, L2gradient=True)
# axs[4].imshow(edges)


plt.show()

In [ ]:
def mixed_pipeline(img_p, apply_erode=False):
    def gamma_correction(image, gamma):
        image = image / 255.0
        corrected = np.power(image, gamma)
        return np.uint8(corrected * 255.0)
    
    c_img, og_img, cvt_img, enhanced_img, img_dilate = get_enhanced_image(img_p, apply_dilation=True)
    red_green = og_img[...,:2].mean(axis=-1)
    red_green = cv2.GaussianBlur(red_green, (11, 11), 5)
    gamma_rg = gamma_correction(red_green, gamma=2.0)
    gamma_rg_int = (gamma_rg > 20).astype(np.uint8) * 255
    
    # blurred = cv2.GaussianBlur(enhanced_img, (11, 11), 5)
    # blurred = cv2.GaussianBlur(gamma_rg, (11, 11), 5)
    thresh = cv2.adaptiveThreshold(gamma_rg, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,21, 5)
    
    if apply_erode:
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))  # Adjust kernel size as needed
        thresh = cv2.erode(thresh, kernel, iterations=2)

    _, labels, bboxes, _ = cv2.connectedComponentsWithStats(gamma_rg_int, connectivity=8)

    
    for box in bboxes[1:]:
        x,y,w,h,area = box
        if area > 5000:
            thresh[y + h:og_img.shape[0],x-30:x+w+30] = 0

    return (bboxes, thresh, og_img)
    
    # plt.imshow(og_img)
    # plt.imshow(thresh, cmap='gray',alpha=0.5)
    
    # for box in bboxes[1:]:
    #     x,y,w,h,area = box
    #     if area > 5000:
    #         plt.gca().add_patch(patches.Rectangle((x,y),w,h,edgecolor='red', facecolor='none'))

    # plt.show()


In [ ]:
# idx = randint(0, img_paths.__len__()-1)
# print(idx)
# mixed_pipeline(img_p=img_paths[idx])

In [ ]:
roi_extracted_imgs = []

n = img_paths.__len__()
grid_shape = (int(np.sqrt(n))+1, int(np.sqrt(n))+1)

for i, img_p in tqdm_notebook(enumerate(img_paths)):
    bboxes, thresh, og_img = mixed_pipeline(img_p=img_p)
    roi_extracted_img = np.zeros_like(og_img)
    for box in bboxes[1:]:
        x,y,w,h,area = box
        if area > 10000:
            # ax.add_patch(patches.Rectangle((x-10,0),w+20,h+y,edgecolor='red', facecolor='none'))
            roi_extracted_img[0:h+y, x-15:x+w+30] = og_img[0:h+y, x-15:x+w+30]
    
    roi_extracted_imgs.append(roi_extracted_img)

In [ ]:
cvt_img = cv2.cvtColor(roi_extracted_imgs[0], cv2.COLOR_RGB2LAB)
gamma_cvt_img = gamma_correction(cv2.convertScaleAbs(cvt_img[...,0], alpha=2.0,beta=100), gamma=2.5)

fig, axs = plt.subplots(1,2)
axs[0].imshow(gamma_cvt_img,cmap='gray')
axs[1].imshow(cvt_img[...,0],cmap='gray')
plt.show()

In [ ]:
# plt.imshow(np.clip(roi_extracted_imgs[0][...,0] * roi_extracted_imgs[0][...,1],a_max=255, a_min=0))
idx = 21
i = gamma_correction(cv2.convertScaleAbs(roi_extracted_imgs[idx][...,0] * roi_extracted_imgs[idx][...,1], alpha=2.0,beta=100), gamma=2.0)
min_max_scale = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))

fig, axs = plt.subplots(1,2)
axs[0].imshow(roi_extracted_imgs[idx])
axs[1].imshow(min_max_scale(cv2.GaussianBlur(i,(11,11), sigmaX=9, sigmaY=9))>0.3)

In [ ]:
# plt.imshow()

test_img = roi_extracted_imgs[0]
test_img = min_max_scale(test_img)
plt.imshow(test_img[...,0] + test_img[...,1])
plt.show()
plt.imshow(test_img[...,0])

In [ ]:
import imageio
import warnings
warnings.filterwarnings('ignore')

In [ ]:
mask_paths = glob.glob('../../../dataset/Seek Thermal/masks/*.png')
new_img_paths = ['D:/semester 1 study/Goettingen study material/Practical Course Data Fusion/dataset/Seek Thermal/jpegs/'+x.split('\\')[-1].replace('png','jpg') for x in mask_paths]

In [ ]:
mask = imageio.imread(mask_paths[0])
mask = mask == 2

plt.imshow(mask)

In [ ]:
idx = 2
_, axs = plt.subplots(1,3)
for i in range(3):
    # axs[i].imshow(og_img[...,i])
    axs[i].imshow(roi_extracted_imgs[idx][...,i])
    axs[i].axis('off')
plt.title('ORIGINAL')
    
for color, color_conv in color_convs.items():
    # cvt_img = cv2.cvtColor(og_img, color_conv)
    cvt_img = cv2.cvtColor(roi_extracted_imgs[idx], color_conv)
    _, axs = plt.subplots(1,3)
    for i in range(3):
        axs[i].imshow(cvt_img[...,i], cmap='gray')
        axs[i].axis('off')
    plt.title(color)

In [ ]:
idx = 20

yuv_img = cv2.cvtColor(roi_extracted_imgs[idx], cv2.COLOR_RGB2YUV)

fig, axs = plt.subplots(1,3)
axs[0].imshow(roi_extracted_imgs[idx][...,0])
axs[1].imshow(yuv_img[...,-1], cmap='gray', alpha=0.5)
axs[2].imshow(yuv_img[...,-1], cmap='gray', alpha=0.5)
plt.show()

In [ ]:
plt.imshow(roi_extracted_imgs[idx][...,0] * yuv_img[...,-1])

In [ ]:
from random import randint
idx = randint(0, img_paths.__len__()-1)
og_img = utils.rotate_to_vertical(img_paths[idx])
print(idx)
cvt_img = cv2.cvtColor(og_img,cv2.COLOR_RGB2HSV)

gam_cvt_img = gamma_correction(cv2.convertScaleAbs(cvt_img[...,-1], alpha=1.0,beta=-20), gamma=2.0)

adap_thresh = cv2.adaptiveThreshold(gam_cvt_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 7)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))  # Adjust kernel size as needed
img_dilate = cv2.dilate(adap_thresh, kernel, iterations=3)

fig, axs = plt.subplots(1,2)
axs[0].imshow(gam_cvt_img,cmap='gray')
axs[0].axis('off')
axs[1].imshow(img_dilate,cmap='gray')
axs[1].axis('off')
plt.show()

In [ ]:

adap_thresh = cv2.adaptiveThreshold(gam_cvt_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 31, 7)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # Adjust kernel size as needed
img_dilate = cv2.dilate(adap_thresh, kernel, iterations=3)
# cv2.dilate(adap_thresh, )
plt.imshow(img_dilate)


In [ ]:
t_img = get_threshold_img(gam_cvt_img, apply_open=True)

In [ ]:
plt.imshow(t_img)

In [ ]:
bboxes = get_contours_n_bbox(t_img)

In [ ]:
plt.imshow(og_img)
for bbox in bboxes:
    plt.gca().add_patch(patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], facecolor='none', edgecolor='red'))
    
plt.show()

In [ ]:
_, axs = plt.subplots(1,3)
for i in range(3):
    # axs[i].imshow(og_img[...,i])
    axs[i].imshow(og_img[...,i])
    axs[i].axis('off')
plt.title('ORIGINAL')
    
for color, color_conv in color_convs.items():
    # cvt_img = cv2.cvtColor(og_img, color_conv)
    converted_img = cv2.cvtColor(og_img, color_conv)
    _, axs = plt.subplots(1,3)
    for i in range(3):
        axs[i].imshow(converted_img[...,i])
        axs[i].axis('off')
    plt.title(color)